# Online Advertising Performance Analysis

This notebook analyzes online advertising performance for a 3‑month period (Apr–Jun 2020).
It explores trends, calculates key metrics (CPC, RPC, CR, ROI), and answers business questions
around **campaigns**, **engagement**, **banner sizes**, and **placements**.

> Tip: Replace the sample dataset path with your actual CSV/Excel to re-run this analysis.


In [ ]:
# Imports & Settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 100)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

# IMPORTANT: charts rule -> matplotlib only, no seaborn; one figure per chart; no explicit colors


In [ ]:
# Load Data
# Provide your dataset path if different (CSV with columns described in README).
DATA_PATH = r"/mnt/data/online_ads_project/data/online_ads_sample.csv"

df = pd.read_csv(DATA_PATH, parse_dates=["Day"])
print("Rows:", len(df))
df.head()


In [ ]:
# Feature Engineering
df['Clicks'] = df['Clicks'].clip(lower=0)
df['Cost'] = df['Cost'].clip(lower=0.0)
df['Revenue'] = df['Revenue'].clip(lower=0.0)
df['Post Click Conversions'] = df['Post Click Conversions'].clip(lower=0)
df['Post Click Sales Amount'] = df['Post Click Sales Amount'].clip(lower=0.0)

# Derived metrics
df['CPC'] = np.where(df['Clicks']>0, df['Cost'] / df['Clicks'], np.nan)
df['RPC'] = np.where(df['Clicks']>0, df['Revenue'] / df['Clicks'], np.nan)
df['CR']  = np.where(df['Clicks']>0, df['Post Click Conversions'] / df['Clicks'], np.nan)
df['AOV'] = np.where(df['Post Click Conversions']>0, df['Post Click Sales Amount'] / df['Post Click Conversions'], np.nan)
df['ROI'] = np.where(df['Cost']>0, (df['Revenue'] - df['Cost']) / df['Cost'], np.nan)

# Calendar features
df['weekday'] = df['Day'].dt.day_name()
df['is_weekend'] = df['weekday'].isin(['Saturday', 'Sunday'])

df.describe(include='all')


In [ ]:
# Q1: Overall trend in user engagement throughout the period
eng_trend = df.groupby(['Day','User Engagement']).size().unstack(fill_value=0)
eng_trend_rolling = eng_trend.rolling(7, min_periods=1).mean()

eng_trend_rolling.plot(figsize=(9,5), title='User Engagement Volume (7‑day rolling count)')
plt.xlabel('Day')
plt.ylabel('Count of Records')
plt.show()

eng_trend.tail(10)


In [ ]:
# Q2: How does banner size impact clicks?
banner_clicks = df.groupby('Banner')['Clicks'].sum().sort_values(ascending=False)
print(banner_clicks)
banner_clicks.plot(kind='bar', figsize=(7,4), title='Total Clicks by Banner Size')
plt.xlabel('Banner')
plt.ylabel('Clicks')
plt.show()


In [ ]:
# Q3: Which placements yielded the highest displays and clicks?
place_perf = df.groupby('Placement')[['Displays','Clicks']].sum().sort_values('Displays', ascending=False)
place_perf, place_perf.head(10)


In [ ]:
# Q4: Correlation between Cost and Revenue
corr = df[['Cost','Revenue','Clicks','Displays']].corr()
print(corr)

plt.figure(figsize=(6,4))
plt.scatter(df['Cost'], df['Revenue'])
plt.title('Cost vs Revenue')
plt.xlabel('Cost (USD)')
plt.ylabel('Revenue (USD)')
plt.show()


In [ ]:
# Q5: Average revenue per click (RPC)
avg_rpc = df['RPC'].mean(skipna=True)
print(f"Average RPC: ${avg_rpc:0.2f}")


In [ ]:
# Q6: Campaigns with highest conversion rates (CR)
camp_cr = df.groupby('Campaign').apply(lambda g: (g['Post Click Conversions'].sum() / g['Clicks'].sum()) if g['Clicks'].sum()>0 else np.nan)
camp_cr = camp_cr.sort_values(ascending=False)
camp_cr


In [ ]:
# Q7: Trends in post-click sales amounts over time
daily_sales = df.groupby('Day')['Post Click Sales Amount'].sum()
daily_sales.rolling(7, min_periods=1).mean().plot(figsize=(9,4), title='Post-Click Sales Amount (7‑day rolling sum)')
plt.xlabel('Day'); plt.ylabel('Sales Amount (USD)'); plt.show()
daily_sales.tail(10)


In [ ]:
# Q8: How engagement varies across banner sizes
eng_banner = df.groupby(['Banner','User Engagement'])[['Clicks','Post Click Conversions']].sum()
eng_banner['CR'] = np.where(eng_banner['Clicks']>0, eng_banner['Post Click Conversions']/eng_banner['Clicks'], np.nan)
eng_banner.sort_values('CR', ascending=False)


In [ ]:
# Q9: Placement types with highest post-click conversion rates
place_cr = df.groupby('Placement').apply(lambda g: (g['Post Click Conversions'].sum()/g['Clicks'].sum()) if g['Clicks'].sum()>0 else np.nan)
place_cr.sort_values(ascending=False)


In [ ]:
# Q10: Seasonal patterns or fluctuations (weekday vs weekend)
dow = df.groupby('weekday')[['Displays','Clicks']].sum().reindex([
    'Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'
])
dow.plot(kind='bar', figsize=(9,4), title='Displays & Clicks by Day of Week')
plt.xlabel('Day of Week'); plt.ylabel('Totals'); plt.show()
dow


In [ ]:
# Q11: Is there a correlation between engagement levels and revenue?
# We'll compute average RPC and CR by engagement, and total revenue share.
eng_rev = df.groupby('User Engagement').agg(
    total_revenue=('Revenue','sum'),
    total_clicks=('Clicks','sum'),
    rpc=('RPC','mean'),
    cr=('CR','mean')
)
eng_rev['revenue_share'] = eng_rev['total_revenue'] / eng_rev['total_revenue'].sum()
eng_rev.sort_values('total_revenue', ascending=False)


In [ ]:
# Q12: Outliers detection using Z-scores
from scipy.stats import zscore

num_cols = ['Cost','Clicks','Revenue']
z = df[num_cols].apply(lambda s: (s - s.mean())/s.std(ddof=0))
outliers = df[(z.abs() > 3).any(axis=1)][['Day','Campaign','User Engagement','Banner','Placement'] + num_cols]
print(f"Outlier rows: {len(outliers)}")
outliers.head(20)


In [ ]:
# Q13: Effectiveness by ad size and placement (ROI)
eff = df.groupby(['Banner','Placement']).agg(
    cost=('Cost','sum'),
    revenue=('Revenue','sum'),
    clicks=('Clicks','sum'),
    conversions=('Post Click Conversions','sum')
)
eff['ROI'] = np.where(eff['cost']>0, (eff['revenue']-eff['cost'])/eff['cost'], np.nan)
eff['CR'] = np.where(eff['clicks']>0, eff['conversions']/eff['clicks'], np.nan)
eff.sort_values('ROI', ascending=False).head(15)


In [ ]:
# Q14: Consistent outperformers (ROI mean & variability by Campaign/Banner)
group = df.groupby(['Campaign','Banner']).agg(roi_mean=('ROI','mean'), roi_std=('ROI','std'), n=('ROI','count'))
group = group[group['n']>=10].sort_values('roi_mean', ascending=False)
group.head(20)


In [ ]:
# Q15: Distribution of post-click conversions across placements
conv_dist = df.groupby('Placement')['Post Click Conversions'].sum().sort_values(ascending=False)
conv_dist.plot(kind='bar', figsize=(7,4), title='Conversions by Placement')
plt.xlabel('Placement'); plt.ylabel('Conversions'); plt.show()
conv_dist


In [ ]:
# Q16: Engagement differences weekdays vs weekends
ew = df.groupby(['is_weekend','User Engagement'])['Clicks'].sum().unstack(fill_value=0)
ew


In [ ]:
# Q17: CPC variation across campaigns & banners
cpc_viz = df.groupby(['Campaign','Banner'])['CPC'].mean().unstack()
cpc_viz


In [ ]:
# Q18: Cost per conversion (lower is better)
df['CostPerConversion'] = np.where(df['Post Click Conversions']>0, df['Cost']/df['Post Click Conversions'], np.nan)
cpcv = df.groupby(['Campaign','Placement'])['CostPerConversion'].mean().sort_values()
cpcv.head(20)


In [ ]:
# Q19: Trends in conversion rates by day of week
dow_cr = df.groupby('weekday').apply(lambda g: (g['Post Click Conversions'].sum()/g['Clicks'].sum()) if g['Clicks'].sum()>0 else np.nan)
dow_cr = dow_cr.reindex(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
dow_cr.plot(kind='bar', figsize=(7,4), title='Conversion Rate by Day of Week')
plt.xlabel('Day'); plt.ylabel('CR'); plt.show()
dow_cr


In [ ]:
# Q20: Effectiveness across engagement types (post-click conversions)
eng_eff = df.groupby('User Engagement').agg(
    clicks=('Clicks','sum'),
    conversions=('Post Click Conversions','sum'),
    cr=lambda g: g['Post Click Conversions'].sum()/g['Clicks'].sum()
).sort_values('cr', ascending=False)
eng_eff


In [ ]:
# Summary KPIs
summary = {
    'Total Displays': int(df['Displays'].sum()),
    'Total Clicks': int(df['Clicks'].sum()),
    'Total Cost ($)': float(df['Cost'].sum()),
    'Total Revenue ($)': float(df['Revenue'].sum()),
    'Total Conversions': int(df['Post Click Conversions'].sum()),
    'Avg CPC ($)': float(df['CPC'].mean(skipna=True)),
    'Avg RPC ($)': float(df['RPC'].mean(skipna=True)),
    'Overall CR': float(df['Post Click Conversions'].sum()/df['Clicks'].sum()) if df['Clicks'].sum()>0 else np.nan,
    'Overall ROI': float((df['Revenue'].sum()-df['Cost'].sum())/df['Cost'].sum()) if df['Cost'].sum()>0 else np.nan
}
summary
